# Алгоритмы сортировки

Пусть на некотором множестве элементов задано
отношение *полного порядка* $\le$, т.е.

$$
\begin{gathered}
a \le b ~~\&~~ b \le c ~~\Rightarrow~~ a \le c\\
\forall a, b ~\rightarrow~ a \le b  ~\text{или}~ b \le a\\
a \le b  ~~\&~~ b \le a  ~~\equiv~~ a = b
\end{gathered}
$$

Задача сортировки упорядоченной последовательности элементов такого множества
$\langle a_1, a_2, \dots, a_n\rangle$
сводится к поиску перестановки
$\langle a'_1, a'_2, \dots, a'_n\rangle$
такой что $a'_1 \le a'_2 \le \dots \le a'_n$

Элементами множества могут быть не только числа, но и произвольные объекты.
На практике для введения на них отношения полного порядка
можно задать *функцию сравнения*, например

```python
>> cmp(a, b)  #  <  0   если a < b
>> cmp(a, b)  #  == 0   если a == b
>> cmp(a, b)  #  >  0   если a > b
```

Другой вариант – ввести функцию отображения множества объектов на множество
ключей, где уже задан полный порядок. Именно такой такая функция используется в Python.

Алгоритмы сортировки, которые оперируют лишь этими абстрактными функциями и не зависят от типа элементов,
называют сортировками *на основе сравнений*.

Далее в описании алгоритмов в качестве упорядоченной последовательности будет фигурировать массив.

# Сортировка пузырьком
Bubble sort (Knuth, Section 5.2.2)

Один из простейших способов сортировки состоит следующем:
* Сравним $a_1$ и $a_2$, поменяем их местами если первый элемент больше второго.
* Сделаем то же самое со всеми остальными парами $a_{i}$ и $a_{i+1}$.
* Будем повторять процедуру пока не будет сделано ни одного обмена -- это будет означать, что массив отсортирован.

In [1]:
def bubble_sort(A):
    n = len(A)
    unsorted = True
    
    while unsorted:
        unsorted = False
        for i in range(n-1):
            if A[i] > A[i+1]:
                A[i], A[i+1] = A[i+1], A[i]
                unsorted = True
    return A

В результате каждого обмена бОльшие элементы перемещаются к концу массива («всплывают»). В худшем случае, когда массив отсортирован в обратном порядке, за одну итерацию по всему массиву «всплывает» лишь один такой элемент. Таких проходов будет $n$, и каждый раз мы совершаем $n-1$ операцию сравнения, т.е. суммарно $n(n-1) = O(n^2)$.

# Сортировка вставками

Insertion sort (Cormen, Section 2.1)

При сортировке вставками мы мысленно выделяем в массиве отсортированный префикс, и постепенно увеличиваем его размер.
* На первом шаге таким префиксом можно считать $\langle a_1 \rangle$ (тривиально отсортирован).
* На $i$-м шаге у нас будет отсортированый префикс $\langle a'_1, ..., a'_{i}\rangle$, к которому мы добавляем $a_{i+1}$, перемещая его справа налево, пока он не встанет на корректную позицию, получая в итоге $(i+1)$-элементный отсортированый префикс.

In [2]:
def insertion_sort(A):
    for j in range(1, len(A)):
        # В этой точке отсортированый префикс в массиве
        # имеет длину j
        key = A[j]
        i = j
        while i>0 and A[i-1]>key:
            A[i] = A[i-1]
            i -= 1
        A[i] = key
    return A

В худшем случае (отсортированный в обратном порядке массив) элемент $a_{i+1}$ будет «протащен» через все $i$ элементов префикса, количество операций при этом будет пропорционально
$$T(n) = \sum_{i=1}^{n} i = \frac{n(n-1)}{2}$$
и хотя $T(n)$ по-прежнему $O(n^2)$, количество операций здесь меньше чем в сортировке пузырьком на постоянный множитель.


# Сложность сортировок на основе сравнений

Для сортировок на основе сравнений можно аналитически оценить *минимально необходимое* в худшем случае количество операций сравнения.

Пусть последовательность содержит $n$ различных элементов. Операция сравнения имеет два исхода, при которых алгоритм преобразует исходную перестановку элементов в одну из двух других. Таким образом $f(n)$ сравнений потенциально могут «покрыть» $2^{f(n)}$ возможных перестановок. Но отсортированная последовательность может соответствовать любой из $n!$ возможных перестановок, поэтому

$$
2^{f(n)} \ge n!  ~~\Rightarrow~~ f(n) \ge \log_2{(n!)}
$$

Из формулы Стирлинга:
$$
\log_2{(n!)} = n\log_2{n} - (\log_2{e})n + O(\log n) \in \Theta(n \log_2 n)\\
$$

Значит как минимум $f(n) \in \Omega(n \log_2 n)$.

# «Разделяй и властвуй»

Divide and Conquer (Cormen, Section 2.3.1, Chapter 4).

Следующие два алгоритма сортировки объединяет общая парадигма построения алгоритмов: *разделяй и властвуй*. Такие алгоритмы можно разбить на три части:
* **Divide:** разделение исходной задачи на несколько подзадач того же типа, но меньшего размера.
* **Conquer:** рекурсивно применять алгоритм к этим подзадачам, пока их размер не станет достаточно мал чтобы решить их за константное время.
* **Combine:** объединить решения подзадач в решение исходной задачи.

# Сортировка слиянием
Merge Sort (Cormen, Section 2.3.1)

Сортировка слиянием основана на алгоритме объединения («слияния») двух отсортированых массовов в один отсортированый.

TODO

# Master Theorem

Пусть $a \ge 1$ и $b \ge 1$ -- константы, $f(n)$ некоторая функция, а $T(n)$ определена на $\mathbb{N}$ рекуррентной формулой:
$$
T(n) = aT\left(\frac{n}{b}\right) + f(n)
$$

Тогда $T(n)$ может иметь следующие асимптотические оценки:
* Если $f(n) = O(n^{\log_b{a-\epsilon}})$ при некотором $\epsilon > 0$, то $T(n) \in \Theta(n^{\log_b{a}})$
* Если $f(n) = \Theta(n^{\log_b{a}})$ то $T(n) \in \Theta(n^{\log_b{a}} \log{n})$
* Если $f(n) = \Omega(n^{\log_b{a+\epsilon}})$ при некоторого $\epsilon > 0$, и если $af(n/b) \le cf(n)$ при некотором $c<1$, то $T(n) \in \Theta(f(n))$.

Сортировка слиянием:
$$
T(n) = 2T(n/2) + \Theta(n) = \Theta(n^{\log_2{2}} \log{n}) = \Theta(n\log{n})
$$

Умножение Карацюбы:
$$
T(n) = 3T(\lceil n/2 \rceil) + \Theta(n) = \Theta(n^{\log_{2} 3})
$$

# «Быстрая» сортировка
Quicksort (Cormen, Chapter 7)

Quicksort тоже использует парадигму «разделяй и властвуй». В in-place варианте алгоритм состоит из следующих шагов:

* Выберем в массиве некоторый элемент. Он называется *опорным* (pivot).
* Расположим опорный элемент в массиве так, чтобы слева от него оказались меньшие элементы, а справа — большие.
* Отсортируем левый и правый подмассивы.

Используя для передача подмассивов индексы их начала (`lo`) и конца (`hi`), рекурсивная часть будет выглядеть так:

In [6]:
def quicksort(A):
    return qs(A, 0, len(A)-1)

def qs(A, lo, hi):
    if lo < hi:
        p = partition(A, lo, hi)
        qs(A, lo, p-1)
        qs(A, p+1, hi)
    return A

Основная работа происходит в функции ``partition``.
Существует несколько вариантов её реализации.
В самом простом из них последний элемент массива берётся в качестве опорного:

In [7]:
def partition(A, lo, hi):
    pivot = A[hi]
    i = lo
    
    for j in range(lo, hi):
        if A[j] <= pivot:
            A[i], A[j] = A[j], A[i]
            i += 1

    A[i], A[hi] = A[hi], A[i]
    return i

# Поиск порядковых статистик

Selection problem (Cormen, Chapter 9)

$i$-й *порядковой статистикой* множества из $n$ элементов называется $i$-й по величине элемент множества. Например, 1-я порядковая статистика — минимальный элемент множества. *Медианой* будем называть $i$-ю порядковую статистику где $i=\lfloor(n+1)/2\rfloor$.

Очевидный способ поиска $i$-й порядковой статистики в массиве: отсортировать элементы по возрастанию и взять элемент по индексу $(i-1)$. Это корректное решение, но оно требует как минимум $\Omega(n\log{n})$ операций.

TODO

# Сортировка выбором

Selection sort

Рассмотрим ещё одну «медленную» сортировку. Она похожа на сортировку вставками, но теперь в отсортированный префикс мы будем добавлять не следующий элемент, а минимальный из оставшихся. На $i$-м шаге у нас будет отсортированый префикс $\langle a'_1, ..., a'_{i}\rangle$, к которому мы добавляем $a_{min}= \min \langle a_{i+1}, ..., a_{n}\rangle$ меняя $a_{min}$ и $a_{i+1}$ местами.

In [8]:
def selection_sort(A):
    n = len(A)
    for i in range(n):
        min_i = i
        for j in range(i, n):
            if A[j] < A[min_i]:
                min_i = j  
        A[i], A[min_i] = A[min_i], A[i]
    return A

Поиск минимума по $n, n-1, n-2, \dots$  элементам суффикса массива здесь производится $n$ раз, т.е. снова имеем квадратичное число операций. Аналогично реализуется алгоритм, в котором строится отсортированный суффикс, и вместо минимума ищется максимум.

Если бы поиск минимума или максимума производился за сублинейное время, то и сама сортировка работала бы быстрее. Построению такого алгоритма посвящены следующие два раздела.

# Очередь с приоритетом и двоичная куча
Priority Queue and Binary Heap

*Очередь с приоритетом* – абстрактный тип данных поддерживающий добавление элементов, а также удаление и/или поиск максимального или минимального элемента.

Наиболее эффективную реализацию очереди с приоритетом даёт структура данных *двоичная куча*. Её можно рассматривать *почти полное* бинарное дерево: в нём должны быть заполнены все уровни кроме разве что нижнего, который может быть частично заполнен слева направо.

Элементы такого дерева можно хранить в массиве, в порядке обхода уровней слева направо начиная с корня. В таком массиве можно быстро находить родительские и дочерние узлы:

In [9]:
parent = lambda i: (i-1)//2
left  =  lambda i: 2*i+1
right =  lambda i: 2*i+2

Заметим также, что любой префикс этого массива будет являться деревом кучи меньшего размера.

В дереве кучи должно выполняться одно из двух свойств:
* **max-heap:** значение любого узла больше значений его дочерних узлов
* **min-heap:** значение любого узла меньше значений его дочерних узлов

В дальнейшем мы будем рассматривать только max-heap. Необходимо построить in-place алгоритм,
который трансформировал бы произвольный массив в max-heap.

Рассмотрим поддерево из трёх элементов на предпоследнем уровне. Для превращения его в кучу надо выбрать из этих элементов максимальный и переместить в корень поддерева. Аналогично поступаем с поддеревьями уровнем выше, но теперь дополнительно рекурсивно вызываем процедуру для нижнего поддерева куда переместился меньший элемент, на случай если это действие нарушило свойство кучи.

Итоговая процедура, запускаемая для некоторого поддерева с корнем по индексу `i`:

In [12]:
def float_down(A, i, n):
    l, r = left(i), right(i)
    largest = i
    if l < n and A[l] > A[largest]: largest = l
    if r < n and A[r] > A[largest]: largest = r
    
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        float_down(A, largest, n)

Чтобы построить max-heap целиком, нужно вызвать её последовательно для всех поддеревьев начиная с предпоследнего уровня:

In [13]:
def heapify(A, n):
    for i in range(n//2, -1, -1):
        bubble_up(A, i, n)
    return A

TODO

# Пирамидальная сортировка
Heapsort

TODO

In [14]:
def heapsort(A):
    n = len(A)
    heapify(A, n)
    for i in range(n-1, 0, -1):
        A[0], A[i] = A[i], A[0]
        n -= 1
        float_down(A, 0, n)
    return A